In [ ]:
print("Hola Spark desde VS Code 🚀")

In [1]:
import sys
sys.path.append("/scripts/config")
from db_config import db_config

print("👉 JDBC URL:", db_config["jdbc_url"])
print("👉 Usuario:", db_config["user"])


👉 JDBC URL: jdbc:sqlserver://host.docker.internal:1433;databaseName=olva;encrypt=false;trustServerCertificate=true
👉 Usuario: etl_user


In [ ]:
# ===== Diagnóstico rápido de entorno/red/Spark =====
import os, sys, socket, platform

print("Python:", sys.version)
print("SO:", platform.platform())
print("HOSTNAME:", socket.gethostname())
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))

# ¿Está pyspark instalado y qué versión?
try:
    import pyspark
    print("pyspark.__version__:", pyspark.__version__)
except Exception as e:
    print("pyspark import ERROR:", e)

# ¿Resuelve y conecta a spark-master:7077 y namenode:8020?
for host, port in [("spark-master", 7077), ("namenode", 8020)]:
    try:
        s = socket.create_connection((host, port), timeout=3)
        print(f"OK conexión a {host}:{port}")
        s.close()
    except Exception as e:
        print(f"FALLO conexión a {host}:{port} → {e.__class__.__name__}: {e}")


In [ ]:
print(spark.sparkContext.getConf().getAll())



In [ ]:
import os
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))

try:
    spark.stop()
    print("Spark anterior detenido")
except:
    print("No había sesión activa")


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("Validacion-Spark-HDFS-Dary")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")
    .config("spark.executor.cores", "2")
    .config("spark.executor.memory", "3g")
    .config("spark.driver.memory", "2g")
    .getOrCreate())

print("✅ Spark version:", spark.version)
print("✅ Master URL:", spark.sparkContext.master)


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("Validacion-Spark-HDFS-Dary")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")
    .config("spark.executor.cores", "2")
    .config("spark.executor.memory", "3g")
    .config("spark.driver.memory", "2g")
    # 🔑 Forzar a Spark a usar los XML de Hadoop
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:8020")
    .getOrCreate())



In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("set hive.metastore.uris").show(truncate=False)

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import col

# DataFrame de prueba
df = spark.createDataFrame([
    Row(id=1, ciudad="Manta", total=185),
    Row(id=2, ciudad="Lima",  total=3.0),
    Row(id=3, ciudad="Quito", total=4.0),
])

# Ruta en HDFS
out_bronze = "hdfs://namenode:8020/bronze/data_df/"

# Escritura en HDFS
(df.coalesce(1)
   .write.mode("overwrite")
   .parquet(out_bronze))

print("✅ Escrito en HDFS →", out_bronze)

# Lectura desde HDFS
df2 = spark.read.parquet(out_bronze)
print("Filas leídas:", df2.count())
df2.orderBy(col("total").desc()).show()



In [30]:
# Leer dataset clientes desde HDFS
clientes = spark.read.parquet("hdfs://namenode:8020/bronze/clientes/")




AnalysisException: 'Unable to infer schema for Parquet. It must be specified manually.;'

In [ ]:




# Mostrar primeras filas
clientes.show(5, truncate=False)


In [ ]:

# Mostrar esquema
clientes.printSchema()




In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, col

# Leer Bronze
clientes = spark.read.parquet("hdfs://namenode:8020/bronze/clientes/")

# Agregar columnas de partición (a partir de CreateTime)
clientes_part = (clientes
    .withColumn("year", year(col("CreateTime")))
    .withColumn("month", month(col("CreateTime")))
    .withColumn("day", dayofmonth(col("CreateTime")))
)

# Guardar en Silver particionado
out_silver = "hdfs://namenode:8020/silver/clientes/"
(clientes_part
    .write
    .mode("overwrite")
    .partitionBy("year", "month", "day")
    .parquet(out_silver))

print("✅ Clientes guardados en Silver particionados por CreateTime →", out_silver)

# Validar
spark.read.parquet(out_silver).show(5, truncate=False)


In [ ]:
import os
from dotenv import load_dotenv

# Verificar si el archivo .env existe
env_path = "/etc/credenciales/.env"
print(f"📁 Verificando archivo: {env_path}")
print(f"📁 Archivo existe: {os.path.exists(env_path)}")

# Forzar recarga del archivo .env
load_dotenv(env_path, override=True)

# Debug: Ver todas las variables de entorno que empiecen con DB_
print("\n🔍 Variables de entorno DB_:")
for key, value in os.environ.items():
    if key.startswith("DB_"):
        print(f"  {key} = {value}")

# Configuración de la base de datos
db_config = {
    "jdbc_url": os.getenv("DB_JDBC_URL"),
    "database": os.getenv("DB_DATABASE"),
    "driver": os.getenv("DB_DRIVER"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "port": os.getenv("DB_PORT")
}

print("\n✅ Config cargada:", db_config)

# Debug adicional: verificar cada variable individualmente
print("\n🔎 Debug individual:")
variables = ["DB_JDBC_URL", "DB_DATABASE", "DB_DRIVER", "DB_USER", "DB_PASS", "DB_PORT"]
for var in variables:
    value = os.getenv(var)
    print(f"  {var}: {value} ({'✅ OK' if value else '❌ MISSING'})")

# Leer archivo .env directamente para debug
try:
    print(f"\n📖 Contenido del archivo {env_path}:")
    with open(env_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for i, line in enumerate(lines, 1):
            # Mostrar caracteres especiales
            line_repr = repr(line.rstrip())
            print(f"  Línea {i}: {line_repr}")
except Exception as e:
    print(f"❌ Error leyendo archivo: {e}")

# Función para cargar manualmente si dotenv falla
def load_env_manual(file_path):
    print(f"\n🔧 Intentando carga manual del archivo...")
    env_vars = {}
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                original_line = line
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    try:
                        key, value = line.split('=', 1)
                        key = key.strip()
                        value = value.strip()
                        env_vars[key] = value
                        os.environ[key] = value
                        print(f"  ✅ Cargada: {key} = {value}")
                    except ValueError as e:
                        print(f"  ❌ Error en línea {line_num}: {repr(original_line.strip())} -> {e}")
                elif line:
                    print(f"  ⚠️  Línea ignorada {line_num}: {repr(original_line.strip())}")
    except Exception as e:
        print(f"❌ Error en carga manual: {e}")
        return {}
    
    return env_vars

# Intentar carga manual para las variables faltantes
print("\n⚠️  Algunas variables faltan, intentando carga manual...")
manual_vars = load_env_manual(env_path)

# Verificar de nuevo después de la carga manual
print("\n🔄 Verificación después de carga manual:")
for var in variables:
    value = os.getenv(var)
    print(f"  {var}: {value} ({'✅ OK' if value else '❌ STILL MISSING'})")

In [ ]:
print(db_config)


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("Validacion-SQLServer")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")
    .config("spark.executor.cores", "2")
    .config("spark.executor.memory", "3g")
    .config("spark.driver.memory", "2g")
    .config("spark.jars", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")  # 👈 necesario
    .getOrCreate())


Exception: Unable to find py4j in /opt/spark/python, your SPARK_HOME may not be configured correctly

In [1]:
df = (spark.read.format("jdbc")
    .option("url", db_config["jdbc_url"])
    .option("dbtable", "dbo.Clientes")   # 👈 ajusta a tu tabla real
    .option("user", db_config["user"])
    .option("password", db_config["password"])
    .option("driver", db_config["driver"])
    .load()
)

df.show(5, truncate=False)



NameError: name 'spark' is not defined

In [ ]:
import sys
sys.path.append("/scripts/config")   # ruta donde vive db_config.py
from db_config import db_config

print(db_config["jdbc_url"])


In [ ]:
print("=== Contenido completo de db_config ===")
for key, value in db_config.items():
    if key == "password":
        print(f"{key}: {'*' * len(str(value))}")  # Ocultar password
    else:
        print(f"{key}: {value}")

In [ ]:
import os
jar_path = "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar"
print(f"JAR exists: {os.path.exists(jar_path)}")

if os.path.exists(jar_path):
    print(f"JAR size: {os.path.getsize(jar_path)} bytes")
    print("✅ El JAR está montado correctamente")
else:
    print("❌ El JAR no está disponible en el contenedor")

In [12]:
from pyspark.sql import SparkSession

# Cerrar sesión actual
try:
    spark.stop()
    print("✅ Sesión anterior cerrada")
except:
    print("ℹ️ No había sesión previa")

# Reiniciar con configuración explícita del JAR
spark = (SparkSession.builder
    .appName("Validacion-Spark-HDFS-Dary")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")
    .config("spark.executor.cores", "2")
    .config("spark.executor.memory", "3g")
    .config("spark.driver.memory", "2g")
    .config("spark.jars", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.driver.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .enableHiveSupport()
    .getOrCreate())

print("✅ Spark version:", spark.version)
print("✅ Master URL:", spark.sparkContext.master)
print("✅ Spark reiniciado con driver JDBC")

ℹ️ No había sesión previa


25/09/10 16:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark version: 2.4.5
✅ Master URL: spark://spark-master:7077
✅ Spark reiniciado con driver JDBC


In [ ]:
import os
jar_path = "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar"
print(f"JAR exists: {os.path.exists(jar_path)}")

if os.path.exists(jar_path):
    print(f"JAR size: {os.path.getsize(jar_path)} bytes")
    print("✅ El JAR está disponible!")
else:
    print("❌ El JAR aún no está disponible")

In [13]:
# Importar configuración
import sys
sys.path.append("/scripts/config")
from db_config import db_config

# Probar conexión
try:
    df_test = (spark.read.format("jdbc")
        .option("url", db_config["jdbc_url"])
        .option("user", db_config["user"])
        .option("password", db_config["password"])
        .option("driver", db_config["driver"])
        .option("query", "SELECT 1 AS test_col")
        .load())
    
    print("✅ ¡Conexión JDBC exitosa!")
    df_test.show()
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

✅ ¡Conexión JDBC exitosa!


[Stage 0:>                                                          (0 + 1) / 1]

+--------+
|test_col|
+--------+
|       1|
+--------+



In [14]:
# Consultar las tablas disponibles (versión simplificada)
try:
    df_tablas = (spark.read.format("jdbc")
        .option("url", db_config["jdbc_url"])
        .option("user", db_config["user"])
        .option("password", db_config["password"])
        .option("driver", db_config["driver"])
        .option("query", """
            SELECT TOP 100
                TABLE_SCHEMA as esquema,
                TABLE_NAME as nombre_tabla,
                TABLE_TYPE as tipo_tabla
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
        """)
        .load())
    
    print("📊 Tablas disponibles en la base de datos 'olva':")
    df_tablas.show(100, truncate=False)
    print(f"\n📈 Total de tablas mostradas: {df_tablas.count()}")
    
except Exception as e:
    print(f"❌ Error consultando tablas: {str(e)}")

📊 Tablas disponibles en la base de datos 'olva':
+-------+------------+----------+
|esquema|nombre_tabla|tipo_tabla|
+-------+------------+----------+
|dbo    |Clientes    |BASE TABLE|
+-------+------------+----------+


📈 Total de tablas mostradas: 1


In [15]:
def read_sql_query(query: str):
    """
    Ejecuta un query en SQL Server usando la configuración JDBC del proyecto.
    Retorna un DataFrame de Spark.
    """
    return (spark.read.format("jdbc")
        .option("url", db_config["jdbc_url"])
        .option("user", db_config["user"])
        .option("password", db_config["password"])
        .option("driver", db_config["driver"])
        .option("query", query)
        .load())


In [40]:
df_clientes = read_sql_query("SELECT TOP 10 * FROM dbo.Clientes")

In [41]:
df_clientes.show()


+---------+-----------+--------------------+----------+--------------------+--------------------+
|ClienteID|     Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|
+---------+-----------+--------------------+----------+--------------------+--------------------+
|        1| Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        2|María López|maria22.nueva@ema...|0987654321|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        3|Carlos Ruiz|carlos.ruiz@email...|0971122334|2025-08-24 21:27:...|                null|
|        4| Ana Torres|ana.torres@email.com|0965544332|2025-08-24 21:27:...|                null|
|        5|Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|
|        7|  leo Pérez| leo.perez@email.com|0991234522|2025-08-29 19:50:...|                null|
+---------+-----------+--------------------+----------+--------------------+--------------------+



In [46]:
df_clientes.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/clientes")


In [44]:
df_clientes.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/cliente_df")


In [47]:
df_validacion = spark.read.parquet("hdfs://namenode:8020/bronze/clientes")
df_validacion.show(5)



+---------+-----------+--------------------+----------+--------------------+--------------------+
|ClienteID|     Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|
+---------+-----------+--------------------+----------+--------------------+--------------------+
|        1| Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        2|María López|maria22.nueva@ema...|0987654321|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        3|Carlos Ruiz|carlos.ruiz@email...|0971122334|2025-08-24 21:27:...|                null|
|        4| Ana Torres|ana.torres@email.com|0965544332|2025-08-24 21:27:...|                null|
|        5|Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|
+---------+-----------+--------------------+----------+--------------------+--------------------+
only showing top 5 rows



In [51]:
from pyspark.sql.functions import max as spark_max
df_clientes = spark.read.parquet("hdfs://namenode:8020/bronze/clientes")
df_clientes.show(5, truncate=False)



+---------+-----------+-----------------------+----------+-----------------------+-----------------------+
|ClienteID|Nombre     |Email                  |Telefono  |CreateTime             |UpdateTime             |
+---------+-----------+-----------------------+----------+-----------------------+-----------------------+
|1        |Juan Pérez |juan.perez@email.com   |089000100 |2025-08-24 21:27:20.383|2025-08-24 21:31:48.536|
|2        |María López|maria22.nueva@email.com|0987654321|2025-08-24 21:27:20.383|2025-08-24 21:31:59.697|
|3        |Carlos Ruiz|carlos.ruiz@email.com  |0971122334|2025-08-24 21:27:20.383|null                   |
|4        |Ana Torres |ana.torres@email.com   |0965544332|2025-08-24 21:27:20.383|null                   |
|5        |Pedro Gómez|pedro.gomez@email.com  |0956677889|2025-08-24 21:27:20.383|null                   |
+---------+-----------+-----------------------+----------+-----------------------+-----------------------+
only showing top 5 rows



In [52]:
df_clientes.agg(spark_max("CreateTime").alias("ultima_fecha_carga")).show()

+--------------------+
|  ultima_fecha_carga|
+--------------------+
|2025-08-29 19:50:...|
+--------------------+



In [67]:
from pyspark.sql import Row
# Crear DF con el watermark
watermark_df = spark.createDataFrame([Row(tabla="clientes",last_watermark="2025-08-29 19:50:42")])

In [68]:
# Guardar en HDFS
watermark_df.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/clientes_watermark")

print("✅ Watermark inicial guardado en /control/clientes_watermark")

✅ Watermark inicial guardado en /control/clientes_watermark


In [69]:
ware_df = spark.read.parquet("hdfs://namenode:8020/bronze/clientes_watermark")
ware_df.show(6)

+-------------------+--------+
|     last_watermark|   tabla|
+-------------------+--------+
|2025-08-29 19:50:42|clientes|
+-------------------+--------+



In [74]:
marca_df= (ware_df
           .filter(ware_df.tabla=="clientes")
           .collect()[0]["last_watermark"])
print("ultima carga de fecha de cliente",marca_df )

ultima carga de fecha de cliente 2025-08-29 19:50:42


In [77]:
def read_sql_query(query: str):
    """
    Ejecuta un query en SQL Server usando la configuración JDBC del proyecto.
    Retorna un DataFrame de Spark.
    """
    return (spark.read.format("jdbc")
        .option("url", db_config["jdbc_url"])
        .option("user", db_config["user"])
        .option("password", db_config["password"])
        .option("driver", db_config["driver"])
        .option("dbtable", query)   # 👈 usar dbtable, no query
        .load())


In [78]:
marca_df= (ware_df
           .filter(ware_df.tabla=="clientes")
           .collect()[0]["last_watermark"])
print("ultima carga de fecha de cliente",marca_df )

ultima carga de fecha de cliente 2025-08-29 19:50:42


In [80]:
# Construir query incremental con alias obligatorio
query_clientes_incr = f"(SELECT * FROM dbo.Clientes WHERE CreateTime > '{marca_df}') as clientes_incr"

# Leer con función corregida
df_incremental = read_sql_query(query_clientes_incr)
df_incremental.show(5, truncate=False)

+---------+---------+-------------------+----------+-----------------------+----------+
|ClienteID|Nombre   |Email              |Telefono  |CreateTime             |UpdateTime|
+---------+---------+-------------------+----------+-----------------------+----------+
|7        |leo Pérez|leo.perez@email.com|0991234522|2025-08-29 19:50:42.572|null      |
|8        |Juan Per |juan.pez@email.com |0991234567|2025-09-10 19:41:11.791|null      |
|9        |Maria L  |ma.lopez@email.com |0987654321|2025-09-10 19:41:11.791|null      |
+---------+---------+-------------------+----------+-----------------------+----------+



In [82]:
df_incremental.write.mode("append").parquet("hdfs://namenode:8020/bronze/clientes")
print("✅ Datos incrementales guardados en Bronze/clientes")

✅ Datos incrementales guardados en Bronze/clientes


In [85]:
df_clie = spark.read.parquet("hdfs://namenode:8020/bronze/clientes")
df_clie.show(10, truncate=False)

+---------+-----------+-----------------------+----------+-----------------------+-----------------------+
|ClienteID|Nombre     |Email                  |Telefono  |CreateTime             |UpdateTime             |
+---------+-----------+-----------------------+----------+-----------------------+-----------------------+
|1        |Juan Pérez |juan.perez@email.com   |089000100 |2025-08-24 21:27:20.383|2025-08-24 21:31:48.536|
|2        |María López|maria22.nueva@email.com|0987654321|2025-08-24 21:27:20.383|2025-08-24 21:31:59.697|
|3        |Carlos Ruiz|carlos.ruiz@email.com  |0971122334|2025-08-24 21:27:20.383|null                   |
|4        |Ana Torres |ana.torres@email.com   |0965544332|2025-08-24 21:27:20.383|null                   |
|5        |Pedro Gómez|pedro.gomez@email.com  |0956677889|2025-08-24 21:27:20.383|null                   |
|7        |leo Pérez  |leo.perez@email.com    |0991234522|2025-08-29 19:50:42.572|null                   |
|7        |leo Pérez  |leo.perez@emai

In [86]:
df_clie.agg(spark_max("CreateTime").alias("ultima_fecha_carga")).show()

+--------------------+
|  ultima_fecha_carga|
+--------------------+
|2025-09-10 19:41:...|
+--------------------+



In [3]:
spark.stop()


NameError: name 'spark' is not defined

In [3]:
!pwd
!ls -l


/home/jovyan
total 4
drwsrwsr-x 2 jovyan users 4096 Oct  9  2022 work


In [4]:
import sys
sys.path.append(r"C:\Users\DARY\Desktop\Dary\Prueba-BigData\Proyecto-hive\scripts")

from parametrossh import init_spark, get_sqlserver_connection


In [9]:
import sys, importlib
sys.path.append(r"C:\Users\DARY\Desktop\Dary\Prueba-BigData\Proyecto-hive\scripts")

import parametrossh
importlib.reload(parametrossh)

from parametrossh import get_params, init_spark, get_sqlserver_connection



In [ ]:
# 1. Iniciar sesión de Spark
spark = init_spark(1, "CargaPedidos")
print("✅ SparkSession inicializada")


✅ SparkSession inicializada


25/09/25 20:13:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/25 20:13:33 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/25 20:13:48 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/25 20:14:03 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [18]:
# 2. Conexión SQL Server
jdbc_url, properties = get_sqlserver_connection(1)

# 3. Leer tabla Pedidos
df_pedidos = spark.read.jdbc(
    url=jdbc_url,
    table="dbo.Pedidos",
    properties=properties
)
print("✅ Pedidos desde SQL Server")
df_pedidos.show(5)

✅ Pedidos desde SQL Server


[Stage 0:>                                                          (0 + 1) / 1]

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
+--------+---------+------+-----------+--------------------+----------+
only showing top 5 rows



25/09/25 20:14:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [9]:
df_pedidos.show(1)

NameError: name 'df_pedidos' is not defined

In [8]:
from pyspark.sql import Row
# Crear DF con el watermark
watermark_df = spark.createDataFrame([Row(tabla="clientes",last_watermark="2025-09-14 19:36:35.439")])


AttributeError: 'NoneType' object has no attribute 'sc'